# Day 09. Exercise 04
# Pipelines and OOP

### Запуск контейнера

docker run -d \\
  --platform linux/amd64 \\
  -p 8888:8888 \\
  -v $(pwd):/home/jovyan/work \\
  --name sklearn \\
  jupyter/scipy-notebook:python-3.8 \\
  bash -c "pip install scikit-learn==0.23.1 tqdm==4.46.1 && start-notebook.sh --NotebookApp.token=''"

#### и выбираем правильный kernel в vscode на localhost (который отдает докер)

In [330]:
import sys
print("Python версия:", sys.version)

import sklearn
print("scikit-learn версия:", sklearn.__version__)

import pandas as pd
print("pandas версия:", pd.__version__)

import numpy as np
print("numpy версия:", np.__version__)

import tqdm
print("tqdm версия:", tqdm.__version__)

Python версия: 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:10) 
[GCC 10.3.0]
scikit-learn версия: 0.23.1
pandas версия: 1.5.0
numpy версия: 1.23.3
tqdm версия: 4.64.1


## 0. Imports

In [331]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import os

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tqdm.notebook import tqdm

# import warnings
# warnings.filterwarnings('ignore')

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).

In [332]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Кастомный трансформер для извлечения признаков из временных меток.
    
    Входные данные: DataFrame с колонками uid, labname, numTrials, timestamp
    Выходные данные: DataFrame с дополнительными колонками hour, weekday (без timestamp)
    """
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        """Метод fit - обязательный для sklearn трансформеров"""
        return self
    
    def transform(self, X):
        """
        Основная логика трансформации:
        - Извлекает час из timestamp
        - Извлекает день недели из timestamp (числа)
        - Удаляет исходную колонку timestamp
        """
        X_copy = X.copy()
        
        # Преобразуем timestamp в datetime если это строки
        if X_copy['timestamp'].dtype == 'object':
            X_copy['timestamp'] = pd.to_datetime(X_copy['timestamp'])
        
        # Извлекаем час (0-23)
        X_copy['hour'] = X_copy['timestamp'].dt.hour
        
        # Извлекаем день недели (0=понедельник, 6=воскресенье)
        X_copy['weekday'] = X_copy['timestamp'].dt.dayofweek
        
        # Удаляем исходную колонку timestamp
        X_copy = X_copy.drop('timestamp', axis=1)
        
        return X_copy

In [333]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    """
    ИСПРАВЛЕННАЯ ВЕРСИЯ - правильно обрабатывает категориальную целевую переменную
    
    Автоматически определяет категориальные колонки и применяет OneHotEncoder,
    исключая целевую переменную из трансформации.
    """
    
    def __init__(self, target_column):
        self.target_column = target_column
        self.categorical_columns = []
        self.encoder = None
        self.encoded_feature_names = []
    
    def fit(self, X, y=None):
        """
        Определяет категориальные колонки и обучает OneHotEncoder
        ИСКЛЮЧАЯ целевую переменную даже если она категориальная
        """
        # Определяем категориальные колонки (исключая целевую)
        self.categorical_columns = []
        for col in X.columns:
            if col != self.target_column:  # исключаем целевую колонку
                # Проверяем является ли колонка категориальной
                if (X[col].dtype == 'object' or 
                    (X[col].dtype in ['int64', 'float64'] and X[col].nunique() <= 50)):
                    self.categorical_columns.append(col)
        
        if self.categorical_columns:
            # Обучаем OneHotEncoder только на признаках
            self.encoder = OneHotEncoder(sparse=False, drop='first')
            self.encoder.fit(X[self.categorical_columns])
            
            # Сохраняем названия новых признаков
            self.encoded_feature_names = self.encoder.get_feature_names(
                self.categorical_columns
            )
        
        return self
    
    def transform(self, X):
        """
        Применяет one-hot кодирование ТОЛЬКО к признакам, НЕ к целевой переменной
        Возвращает X (признаки) и y (целевая переменная) отдельно
        """
        X_copy = X.copy()
        
        # Извлекаем целевую переменную БЕЗ трансформации
        # даже если она категориальная
        y = X_copy[self.target_column].copy()
        
        if self.categorical_columns and self.encoder is not None:
            # Применяем one-hot кодирование только к признакам
            encoded_data = self.encoder.transform(X_copy[self.categorical_columns])
            
            # Создаем DataFrame с закодированными признаками
            encoded_df = pd.DataFrame(
                encoded_data, 
                columns=self.encoded_feature_names,
                index=X_copy.index
            )
            
            # Удаляем исходные категориальные колонки
            X_copy = X_copy.drop(self.categorical_columns, axis=1)
            
            # Добавляем закодированные признаки
            X_copy = pd.concat([X_copy, encoded_df], axis=1)
        
        # Удаляем целевую колонку из признаков
        X_copy = X_copy.drop(self.target_column, axis=1)
        
        return X_copy, y

In [334]:
class TrainValidationTest:
    """
    Класс для разделения данных на тренировочную, валидационную и тестовую выборки.
    """
    
    def __init__(self):
        pass
    
    def split(self, X, y, test_size=0.2, random_state=21):
        """
        Разделяет данные на train/validation/test с стратификацией
        Возвращает X_train, X_valid, X_test, y_train, y_valid, y_test
        """
        # Сначала отделяем test set
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, 
            stratify=y
        )
        
        # Затем из оставшихся данных делаем train/validation
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_temp, y_temp, test_size=test_size, random_state=random_state,
            stratify=y_temp
        )
        
        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [335]:
class ModelSelection:
    """    
    Класс для автоматического выбора лучшей модели среди нескольких алгоритмов.
    Использует GridSearchCV для поиска лучших гиперпараметров каждого алгоритма,
    затем сравнивает их производительность на валидационной выборке.
    """
    
    def __init__(self, grids, grid_dict):
        """
        grids: список GridSearchCV объектов
        grid_dict: словарь {индекс: название_модели}
        """
        self.grids = grids
        self.grid_dict = grid_dict
        self.best_estimators = {}
        self.best_scores = {}
        self.results_df = None
    
    def choose(self, X_train, y_train, X_valid, y_valid):
        """
        Обучает все модели и выбирает лучшую по accuracy на валидации
        """
        results = []
        best_model_name = None
        best_valid_score = -1
        
        for i, grid in enumerate(self.grids):
            model_name = self.grid_dict[i]
            print(f"Estimator: {model_name}")
            
            # Подсчитываем общее количество комбинаций
            total_combinations = 1
            for param_name, param_values in grid.param_grid.items():
                total_combinations *= len(param_values)
            
            with tqdm(total=total_combinations, 
                     bar_format='{percentage:3.0f}%\n{n}/{total} [{elapsed}<{remaining}, {rate_fmt}]') as pbar:
                grid.fit(X_train, y_train)
                pbar.update(total_combinations)
            
            # Получаем результаты
            best_estimator = grid.best_estimator_
            best_params = grid.best_params_
            best_train_score = grid.best_score_
            
            # Оцениваем на валидации
            valid_score = accuracy_score(y_valid, best_estimator.predict(X_valid))
            
            # Сохраняем результаты
            self.best_estimators[model_name] = best_estimator
            self.best_scores[model_name] = valid_score
            
            results.append({
                'model': model_name,
                'params': best_params,
                'valid_score': valid_score
            })
            
            # Отслеживаем лучшую модель
            if valid_score > best_valid_score:
                best_valid_score = valid_score
                best_model_name = model_name
            
            print(f"Best params: {best_params}")
            print(f"Best training accuracy: {best_train_score:.3f}")
            print(f"Validation set accuracy score for best params: {valid_score:.3f} ")
            print()  # Пустая строка между моделями
        
        # Сохраняем результаты в DataFrame
        self.results_df = pd.DataFrame(results)
        
        print(f"Classifier with best validation set accuracy: {best_model_name}")
        
        return best_model_name
    
    def best_results(self):
        """
        Возвращает DataFrame с результатами всех моделей
        """
        if self.results_df is not None:
            # Форматируем params как строки (обрезанные)
            formatted_results = self.results_df.copy()
            formatted_results['params'] = formatted_results['params'].apply(
                lambda x: str(x)[:50] + '...' if len(str(x)) > 50 else str(x)
            )
            return formatted_results[['model', 'params', 'valid_score']]
        else:
            print("Run choose() method first!")
            return None
    
    def get_best_estimator(self, model_name):
        """
        Возвращает лучший estimator для указанной модели
        """
        return self.best_estimators.get(model_name)

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [336]:
class Finalize:
    """    
    Класс для финализации модели - получения финального результата и сохранения.
    """
    
    def __init__(self, estimator):
        self.estimator = estimator
    
    def final_score(self, X_train, y_train, X_test, y_test):
        """
        Обучает модель на всех тренировочных данных и оценивает на тесте
        """
        # Обучаем на полных тренировочных данных
        self.estimator.fit(X_train, y_train)
        
        # Предсказываем на тестовой выборке
        y_pred = self.estimator.predict(X_test)
        
        # Вычисляем accuracy
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"Accuracy of the final model is {accuracy}")
        
        return accuracy
    
    def save_model(self, path):
        """
        Сохраняет модель в указанный путь
        """
        # Создаем директорию если не существует
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        with open(path, 'wb') as f:
            pickle.dump(self.estimator, f)
        
        print(f"Model successfully saved to {path}")

## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

### Загрузка данных

In [337]:
%%time

# 1. Load the data from the file checker_submits.csv
print("=== Загрузка данных... ===\n")
df = pd.read_csv('work/src/data/checker_submits.csv')

print(f"Размер данных: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")
print("\nПервые несколько строк:")
print(df.head())

print(f"\nОбщая информация:")
print(f"- Уникальных пользователей: {df['uid'].nunique()}")
print(f"- Уникальных лабораторных: {df['labname'].nunique()}")
print(f"- Период данных: {df['timestamp'].min()} - {df['timestamp'].max()}")

=== Загрузка данных... ===

Размер данных: (1686, 4)
Колонки: ['uid', 'labname', 'numTrials', 'timestamp']

Первые несколько строк:
      uid   labname  numTrials                   timestamp
0  user_4  project1          1  2020-04-17 05:19:02.744528
1  user_4  project1          2  2020-04-17 05:22:45.549397
2  user_4  project1          3  2020-04-17 05:34:24.422370
3  user_4  project1          4  2020-04-17 05:43:27.773992
4  user_4  project1          5  2020-04-17 05:46:32.275104

Общая информация:
- Уникальных пользователей: 30
- Уникальных лабораторных: 11
- Период данных: 2020-04-17 05:19:02.744528 - 2020-05-21 20:37:00.290491
CPU times: user 42.5 ms, sys: 7.03 ms, total: 49.5 ms
Wall time: 47.6 ms


### Создание и применение preprocessing pipeline

In [338]:
# 2. Create the preprocessing pipeline that consists of 
# two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
print("=== Создание preprocessing pipeline... ===\n")

preprocessing = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('onehot_encoder', MyOneHotEncoder('weekday'))  # weekday как целевая переменная
])

print("Pipeline создан:")
for step_name, step_transformer in preprocessing.steps:
    print(f"  - {step_name}: {step_transformer.__class__.__name__}")

=== Создание preprocessing pipeline... ===

Pipeline создан:
  - feature_extractor: FeatureExtractor
  - onehot_encoder: MyOneHotEncoder


In [339]:
%%time

# 3. Use that pipeline and its method fit_transform() on the initial dataset
print("=== Применение preprocessing pipeline... ===\n")

X, y = preprocessing.fit_transform(df)

print(f"Размер признаков после preprocessing: {X.shape}")
print(f"Размер целевой переменной: {y.shape}")
print(f"Колонки признаков: {X.columns.tolist()}")

print("\nПервые несколько строк признаков:")
print(X.head())

print("\nРаспределение целевой переменной (weekday):")
weekday_counts = y.value_counts().sort_index()
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i, count in enumerate(weekday_counts):
    print(f"{weekdays[i]} (day {i}): {count} samples")

=== Применение preprocessing pipeline... ===

Размер признаков после preprocessing: (1686, 61)
Размер целевой переменной: (1686,)
Колонки признаков: ['numTrials', 'uid_user_1', 'uid_user_10', 'uid_user_11', 'uid_user_12', 'uid_user_13', 'uid_user_14', 'uid_user_15', 'uid_user_16', 'uid_user_17', 'uid_user_18', 'uid_user_19', 'uid_user_2', 'uid_user_20', 'uid_user_21', 'uid_user_22', 'uid_user_23', 'uid_user_24', 'uid_user_25', 'uid_user_26', 'uid_user_27', 'uid_user_28', 'uid_user_29', 'uid_user_3', 'uid_user_30', 'uid_user_31', 'uid_user_4', 'uid_user_6', 'uid_user_7', 'uid_user_8', 'labname_lab02', 'labname_lab03', 'labname_lab03s', 'labname_lab05s', 'labname_laba04', 'labname_laba04s', 'labname_laba05', 'labname_laba06', 'labname_laba06s', 'labname_project1', 'hour_1', 'hour_3', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23']

Пе

### Разделение данных

In [340]:
%%time 

# 4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` 
# using `TrainValidationTest()` and the result of the pipeline.
print("=== Разделение данных на train/validation/test... ===\n")

splitter = TrainValidationTest()
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split(X, y)

print(f"Train set: {X_train.shape[0]} samples")
print(f"Validation set: {X_valid.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

=== Разделение данных на train/validation/test... ===

Train set: 1078 samples
Validation set: 270 samples
Test set: 338 samples
CPU times: user 12.6 ms, sys: 3.02 ms, total: 15.6 ms
Wall time: 14.2 ms


In [341]:
print("Распределение в train set:")
train_dist = y_train.value_counts().sort_index()
for i, count in enumerate(train_dist):
    print(f"{weekdays[i]}: {count}")

print("\nРаспределение в validation set:")
valid_dist = y_valid.value_counts().sort_index()
for i, count in enumerate(valid_dist):
    print(f"{weekdays[i]}: {count}")

print("\nРаспределение в test set:")
test_dist = y_test.value_counts().sort_index()
for i, count in enumerate(test_dist):
    print(f"{weekdays[i]}: {count}")

Распределение в train set:
Monday: 87
Tuesday: 175
Wednesday: 95
Thursday: 253
Friday: 66
Saturday: 174
Sunday: 228

Распределение в validation set:
Monday: 22
Tuesday: 44
Wednesday: 24
Thursday: 63
Friday: 17
Saturday: 43
Sunday: 57

Распределение в test set:
Monday: 27
Tuesday: 55
Wednesday: 30
Thursday: 80
Friday: 21
Saturday: 54
Sunday: 71


### Настройка моделей для GridSearch

In [342]:
# 5. Create an instance of `ModelSelection()`, use the method `choose()` 
# applying it to the models that you want and parameters that you want, 
# get the dataframe of the best results.
print("=== Настройка моделей для GridSearch... ===\n")

# Параметры для SVM (на основе лучших результатов из ex02/ex03)
svm_params = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None],
    'random_state': [21],
    'probability': [True]
}

# Параметры для Decision Tree (на основе лучших результатов из ex02/ex03)
tree_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, 15, 20, 21, 22, 25, None],
    'class_weight': ['balanced', None],
    'random_state': [21]
}

# Параметры для Random Forest (на основе лучших результатов из ex02/ex03)
rf_params = {
    'n_estimators': [30, 50, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [20, 21, 22, 25, None],
    'class_weight': ['balanced', None],
    'random_state': [21]
}

# Создание GridSearchCV объектов
gs_svm = GridSearchCV(
    estimator=SVC(),
    param_grid=svm_params,
    scoring='accuracy',
    cv=2,
    n_jobs=-1
)

gs_tree = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=tree_params,
    scoring='accuracy',
    cv=2,
    n_jobs=-1
)

gs_rf = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=rf_params,
    scoring='accuracy',
    cv=2,
    n_jobs=-1
)

grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}

# Подсчитываем общее количество комбинаций
svm_combinations = np.prod([len(v) for v in svm_params.values()])
tree_combinations = np.prod([len(v) for v in tree_params.values()])
rf_combinations = np.prod([len(v) for v in rf_params.values()])

print(f"SVM: {svm_combinations} комбинаций")
print(f"Decision Tree: {tree_combinations} комбинаций")
print(f"Random Forest: {rf_combinations} комбинаций")
print(f"\nВсего: {svm_combinations + tree_combinations + rf_combinations} комбинаций")

=== Настройка моделей для GridSearch... ===

SVM: 72 комбинаций
Decision Tree: 28 комбинаций
Random Forest: 60 комбинаций

Всего: 160 комбинаций


### Выбор лучшей модели

In [343]:
%%time

# Model selection
print("=== ВЫБОР ЛУЧШЕЙ МОДЕЛИ ===\n")

model_selector = ModelSelection(grids, grid_dict)
best_model_name = model_selector.choose(X_train, y_train, X_valid, y_valid)

=== ВЫБОР ЛУЧШЕЙ МОДЕЛИ ===

Estimator: SVM


  0%
0/72 [00:00<?, ?it/s]

Best params: {'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.733
Validation set accuracy score for best params: 0.815 

Estimator: Decision Tree


  0%
0/28 [00:00<?, ?it/s]

Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': 22, 'random_state': 21}
Best training accuracy: 0.781
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest


  0%
0/60 [00:00<?, ?it/s]

Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'n_estimators': 100, 'random_state': 21}
Best training accuracy: 0.820
Validation set accuracy score for best params: 0.878 

Classifier with best validation set accuracy: Random Forest
CPU times: user 2.44 s, sys: 557 ms, total: 3 s
Wall time: 39.3 s


In [344]:
print("=== РЕЗУЛЬТАТЫ ВСЕХ МОДЕЛЕЙ ===\n")
results_df = model_selector.best_results()
print(results_df.to_string(index=False))

=== РЕЗУЛЬТАТЫ ВСЕХ МОДЕЛЕЙ ===

        model                                                params  valid_score
          SVM {'C': 10, 'class_weight': 'balanced', 'gamma': 'au...     0.814815
Decision Tree {'class_weight': None, 'criterion': 'gini', 'max_d...     0.866667
Random Forest {'class_weight': None, 'criterion': 'gini', 'max_d...     0.877778


### Финализация и сохранение модели

In [345]:
# 6. create an instance of `Finalize()` with your best model, 
# use method `final_score()` and save the model in the format: 
# `name_of_the_model_{accuracy on test dataset}.sav`.
print(f"=== ФИНАЛИЗАЦИЯ С {best_model_name.upper()} ===\n")

best_estimator = model_selector.get_best_estimator(best_model_name)
print(f"Лучшие параметры {best_model_name}:")
for param, value in best_estimator.get_params().items():
    print(f"  {param}: {value}")

=== ФИНАЛИЗАЦИЯ С RANDOM FOREST ===

Лучшие параметры Random Forest:
  bootstrap: True
  ccp_alpha: 0.0
  class_weight: None
  criterion: gini
  max_depth: None
  max_features: auto
  max_leaf_nodes: None
  max_samples: None
  min_impurity_decrease: 0.0
  min_impurity_split: None
  min_samples_leaf: 1
  min_samples_split: 2
  min_weight_fraction_leaf: 0.0
  n_estimators: 100
  n_jobs: None
  oob_score: False
  random_state: 21
  verbose: 0
  warm_start: False


In [346]:
%%time

# Объединяем train и validation для финального обучения
X_train_full = pd.concat([X_train, X_valid], ignore_index=True)
y_train_full = pd.concat([y_train, y_valid], ignore_index=True)

print(f"\nФинальное обучение на {X_train_full.shape[0]} образцах...\n")

finalizer = Finalize(best_estimator)
final_accuracy = finalizer.final_score(X_train_full, y_train_full, X_test, y_test)

# Create model filename with accuracy
model_name_clean = best_model_name.lower().replace(' ', '_')
model_filename = f"work/src/ex04/model/{model_name_clean}_{final_accuracy:.6f}.model"

print(f"\nСохранение модели как: {model_filename}")
finalizer.save_model(model_filename)


Финальное обучение на 1348 образцах...

Accuracy of the final model is 0.8964497041420119

Сохранение модели как: work/src/ex04/model/random_forest_0.896450.model
Model successfully saved to work/src/ex04/model/random_forest_0.896450.model
CPU times: user 332 ms, sys: 22.3 ms, total: 354 ms
Wall time: 356 ms


### Финальный анализ результатов

In [347]:
print("=== ФИНАЛЬНЫЙ АНАЛИЗ ===\n")
print(f"🎯 Лучшая модель: {best_model_name}")
print(f"📊 Финальная точность: {final_accuracy:.6f}")
print(f"💾 Модель сохранена: {model_filename}")
print()

# Анализ предсказаний
y_pred = best_estimator.predict(X_test)

print("📈 Classification Report:")
print(classification_report(y_test, y_pred, target_names=weekdays))

print("\n🔍 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Анализ ошибок по дням недели
print("\n❌ Анализ ошибок по дням недели:")
error_rates = {}

for i, day in enumerate(weekdays):
    if i < len(cm):
        total_samples = sum(cm[i, :])
        correct_predictions = cm[i, i]
        if total_samples > 0:
            error_rate = (total_samples - correct_predictions) / total_samples * 100
            error_rates[day] = error_rate
            print(f"{day}: {error_rate:.2f}% ошибок ({total_samples - correct_predictions}/{total_samples})")

if error_rates:
    worst_day = max(error_rates, key=error_rates.get)
    best_day = min(error_rates, key=error_rates.get)
    print(f"\n🔴 Самый проблемный день: {worst_day} ({error_rates[worst_day]:.2f}% ошибок)")
    print(f"🟢 Лучше всего предсказывается: {best_day} ({error_rates[best_day]:.2f}% ошибок)")

print("\n✅ Pipeline завершен успешно!")

=== ФИНАЛЬНЫЙ АНАЛИЗ ===

🎯 Лучшая модель: Random Forest
📊 Финальная точность: 0.896450
💾 Модель сохранена: work/src/ex04/model/random_forest_0.896450.model

📈 Classification Report:
              precision    recall  f1-score   support

      Monday       0.83      0.70      0.76        27
     Tuesday       0.92      0.87      0.90        55
   Wednesday       0.90      0.87      0.88        30
    Thursday       0.95      0.96      0.96        80
      Friday       0.95      0.86      0.90        21
    Saturday       0.83      0.89      0.86        54
      Sunday       0.88      0.94      0.91        71

    accuracy                           0.90       338
   macro avg       0.89      0.87      0.88       338
weighted avg       0.90      0.90      0.90       338


🔍 Confusion Matrix:
[[19  2  1  1  0  1  3]
 [ 3 48  2  1  0  1  0]
 [ 1  1 26  1  0  1  0]
 [ 0  1  0 77  0  1  1]
 [ 0  0  0  0 18  2  1]
 [ 0  0  0  1  1 48  4]
 [ 0  0  0  0  0  4 67]]

❌ Анализ ошибок по дням недел

# ЗАГРУЗКА МОДЕЛИ И ПРОВЕРКА

In [348]:
print("=== ФИНАЛЬНАЯ ПРОВЕРКА: ЗАГРУЗКА И ТЕСТИРОВАНИЕ МОДЕЛИ ===\n")

# Загружаем сохраненную модель из файла
print(f"Загружаем модель из файла: {model_filename}")
with open(model_filename, 'rb') as f:
    loaded_model_final = pickle.load(f)

print("✅ Модель успешно загружена из файла")

# Создаем новый объект Finalize с загруженной моделью
loaded_finalizer_test = Finalize(loaded_model_final)

print(f"\nИспользуем загруженную модель в классе Finalize...")
print(f"Применяем к тем же тестовым данным: {X_test.shape[0]} образцов\n")

# Модель уже обучена, поэтому мы НЕ вызываем fit()
# Получаем предсказания и score
loaded_predictions = loaded_model_final.predict(X_test)
loaded_final_score = accuracy_score(y_test, loaded_predictions)

print(f"Score загруженной модели: {loaded_final_score}")

# Сравниваем с оригинальным результатом
print(f"\n=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===\n")
print(f"Оригинальный score (до сохранения): {final_accuracy}")
print(f"Score загруженной модели:           {loaded_final_score}")
print(f"Разница:                            {abs(final_accuracy - loaded_final_score)}")

scores_identical = abs(final_accuracy - loaded_final_score) < 1e-15
print(f"\nРезультаты идентичны: {scores_identical}")

if scores_identical:
    print("\n✅ Загруженная модель дает точно такой же результат!")
    print("✅ Сохранение и загрузка модели работают корректно")
else:
    print("\n❌ ОШИБКА: Результаты не совпадают!")
    print("❌ Проблема с сохранением/загрузкой модели")

=== ФИНАЛЬНАЯ ПРОВЕРКА: ЗАГРУЗКА И ТЕСТИРОВАНИЕ МОДЕЛИ ===

Загружаем модель из файла: work/src/ex04/model/random_forest_0.896450.model
✅ Модель успешно загружена из файла

Используем загруженную модель в классе Finalize...
Применяем к тем же тестовым данным: 338 образцов

Score загруженной модели: 0.8964497041420119

=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===

Оригинальный score (до сохранения): 0.8964497041420119
Score загруженной модели:           0.8964497041420119
Разница:                            0.0

Результаты идентичны: True

✅ Загруженная модель дает точно такой же результат!
✅ Сохранение и загрузка модели работают корректно
